In [73]:
import os
import warnings
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [74]:
# Load your data
feature_columns = ['LOS', 'AGE', 'GENDER_M', 'ETHNICITY_Asian', 'ETHNICITY_Black', 'ETHNICITY_Hispanic', 'ETHNICITY_Native_Hawaiian', 'ETHNICITY_Other', 'ETHNICITY_White']
data = pd.read_csv('admit_modified.csv', index_col=False)
data.dropna(subset = feature_columns, inplace = True)
data.reset_index(drop=True, inplace = True)
data['codes'] =  data['PROCEDURE_AND_DIAGNOSIS_ICD'].apply(lambda x: x.replace("'", "")[1:-1].split(", "))
X = data.drop(['MORTALITY_30_DAY'], axis = 1)
y = data['MORTALITY_30_DAY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
X_train.reset_index(drop=True, inplace = True)
X_test.reset_index(drop=True, inplace = True)
y_train.reset_index(drop=True, inplace = True)
y_test.reset_index(drop=True, inplace = True)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(28718, 49) (28718,) (7180, 49) (7180,)


In [75]:
# train skip-gram model
window_size = 71 # max(X_train['codes'].apply(lambda x: len(x))) = 71
skipgram = Word2Vec(vector_size=100, window=window_size, min_count=1, sg=1)
skipgram.build_vocab(data['codes'])
skipgram.train(X_train['codes'], total_examples=skipgram.corpus_count, epochs=skipgram.epochs)
# Extract embeddings
embeddings = {word: skipgram.wv[word] for word in skipgram.wv.index_to_key}


In [76]:
def codes_to_emb(codes):
    return np.mean(np.array([embeddings.get(c) for c in codes]), axis=0)
X_train['embedding'] = X_train['codes'].apply(lambda x: codes_to_emb(x))
X_test['embedding'] = X_test['codes'].apply(lambda x: codes_to_emb(x))
X_train = pd.concat([X_train['embedding'].apply(pd.Series), X_train[feature_columns]], axis=1)
X_test = pd.concat([X_test['embedding'].apply(pd.Series), X_test[feature_columns]], axis=1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(28718, 109) (28718,) (7180, 109) (7180,)


In [77]:


class MetricsCallback(Callback):
    def __init__(self, X_train, y_train, X_test, y_test):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def on_epoch_end(self, epoch, logs=None):
        y_train_pred = self.model.predict(self.X_train)
        y_test_pred = self.model.predict(self.X_test)
        auroc_train = roc_auc_score(self.y_train, y_train_pred)
        auroc_test = roc_auc_score(self.y_test, y_test_pred)
        auprc_train = average_precision_score(self.y_train, y_train_pred)
        auprc_test = average_precision_score(self.y_test, y_test_pred)
        print(f"\nEpoch {epoch+1}: AUROC Train: {auroc_train}, AUROC Test: {auroc_test}, "
              f"AUPRC Train: {auprc_train}, AUPRC Test: {auprc_test}")

def cnn(X_train, y_train, X_test, y_test):
    # Build and compile the neural network
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=109))  # 100 embedding dims + 9 other features
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64))
    model.add(Dense(8))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))  # Suitable for binary classification

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Initialize metrics callback
    metrics_callback = MetricsCallback(X_train, y_train, X_test, y_test)

    # Train the model
    model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[metrics_callback])

    return model

# Example usage
model = cnn(X_train, y_train, X_test, y_test)


Epoch 1/30
225/225 [==============================] - 0s 267us/step

Epoch 1: AUROC Train: 0.7296833044722034, AUROC Test: 0.7123741316299179, AUPRC Train: 0.1024773207564295, AUPRC Test: 0.09638782129398066
898/898 [==============================] - 2s 2ms/step - loss: 0.2521 - accuracy: 0.9479 - val_loss: 0.1869 - val_accuracy: 0.9525
Epoch 2/30
225/225 [==============================] - 0s 321us/step

Epoch 2: AUROC Train: 0.789334237900553, AUROC Test: 0.7778602023327482, AUPRC Train: 0.13528761526577995, AUPRC Test: 0.12917898833697508
898/898 [==============================] - 1s 2ms/step - loss: 0.1917 - accuracy: 0.9525 - val_loss: 0.1723 - val_accuracy: 0.9525
Epoch 3/30
225/225 [==============================] - 0s 302us/step

Epoch 3: AUROC Train: 0.8040725372077919, AUROC Test: 0.7926485968220045, AUPRC Train: 0.14325417932125398, AUPRC Test: 0.138128789049319
898/898 [==============================] - 1s 1ms/step - loss: 0.1819 - accuracy: 0.9525 - val_loss: 0.1670 - val_a